## Data cleaning, in one place

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import warnings
from datetime import datetime
import time
from geopy.distance import great_circle
%matplotlib inline

# This package will output the execution time of each cell. Pretty neat!
warnings.filterwarnings('ignore')
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
warnings.filterwarnings('default')
%load_ext autotime

#### Conglomerating techniques Haris and I used in other notebooks to clean the data. Here we'll do it in one place, for 2015 and 2016.

In [ ]:
training_data_files = ['data/201506-citibike-tripdata.csv', 'data/201507-citibike-tripdata.csv', 'data/201508-citibike-tripdata.csv']
training = pd.DataFrame()
for filename in training_data_files:
    training = training.append(pd.read_csv(filename), ignore_index=True)

In [ ]:
testing_data_files = ["data/201606-citibike-tripdata.csv", "data/201607-citibike-tripdata.csv", "data/201608-citibike-tripdata.csv"]
testing = pd.DataFrame()
for filename in testing_data_files:
    testing = testing.append(pd.read_csv(filename), ignore_index=True)

In [ ]:
# parse time strings of different formats
def parse_date(text):
    try:
        return datetime.strptime(text, '%Y-%m-%d %H:%M:%S')
    except ValueError:
        try:
            return datetime.strptime(text, '%m/%d/%Y %H:%M')
        except ValueError:
            try:
                return datetime.strptime(text, '%m/%d/%Y %H:%M:%S')
            except ValueError:
                raise ValueError('no valid date format found')

In [ ]:
def clean_data(data):
    
    # only focus on subscribing users, the vast majority
    data[data.usertype == 'Subscriber']
    del data['usertype']
    data = data.dropna(how='any').reset_index(drop=True)

    # remove trips of less than 2 minutes
    data = data[data['tripduration'] >= 120]

    # convert birth year to float
    data['birth year'] = pd.to_numeric(data['birth year'])

    # create 'age' attribute, from birth year
    data['age'] = 2015 - data['birth year']
    data['age'] = data['age'].astype(int)
    del data['birth year']

    # change trip duration to minute as integer
    data['tripduration'] = data['tripduration'] / 60
    data['tripduration'] = data['tripduration'].astype(int)

    # no need for bike id
    del data['bikeid']
    
    #get starttimes at array of datetime objects
    starttimes = pd.Series([parse_date(starttime) for starttime in data['starttime']])
    
    # add day of week column and minute column (minutes since 00:00:00 that day)
    data['day of week'] = [starttime.weekday() for starttime in starttimes]
    data['minute'] = [(starttime.hour * 60 + starttime.minute) for starttime in starttimes]
    data = data.dropna(how='any').reset_index(drop=True)
    data['day of week'] = data['day of week'].astype(int)
 
    # calculate round trips
    data['roundtrip'] = pd.Series(calculate_roundtrips(data))
    data['roundtrip'] = data['roundtrip'].astype(int)
    
    return data

In [ ]:
# compute roundtrips
# 0 = one way, 1 = roundtrip
def calculate_roundtrips(trips):
    
    roundtrips = []
    
    # for use with tracking our runtime
    count = 0
    
    # to measure how long this computation takes
    startTime = datetime.now().replace(microsecond=0)
    print("Start time: %s" % startTime)
    
    # loop through entire input dataframe and calculate roundtrips
    for i in trips.index:
        # first let's check if start and stop station are the same (obvious roundtrip)
        if ((trips.loc[i]['start station id'] == trips.loc[i]['end station id']) & 
             (trips.loc[i]['tripduration'] > 8)):
            roundtrips.append(1)
        else:
            # now lets check distance between stations
            start_coordinates = (trips.loc[i]['start station latitude'], 
                                 trips.loc[i]['start station longitude'])
            end_coordinates = (trips.loc[i]['end station latitude'], 
                               trips.loc[i]['end station longitude'])
            distance = great_circle(start_coordinates, end_coordinates).miles
            
            if (distance < .3) & (trips.loc[i]['tripduration'] > 8):
                roundtrips.append(1)
            else:
                roundtrips.append(0)

        # Keep track of how long this calculation is taking
        count += 1
        if (count % 10000 == 0):
            now = datetime.now().replace(microsecond=0)
            print("%d rows processed, %.2f%% done, %s time elapsed" % (
                    count, (count / len(trips)) * 100, now - startTime))
            
    # how long did this take?
    endTime = datetime.now().replace(microsecond=0)
    print("Runtime: %s" % str(endTime - startTime))
    
    return roundtrips

In [ ]:
# created simpler version of clean data
training_simple = training_clean.copy()
del training_simple['start station name']
del training_simple['end station name']
del training_simple['start station latitude']
del training_simple['start station longitude']
del training_simple['end station latitude']
del training_simple['end station longitude']
del training_simple['starttime']
del training_simple['stoptime']

testing_simple = testing_clean.copy()
del testing_simple['start station name']
del testing_simple['end station name']
del testing_simple['start station latitude']
del testing_simple['start station longitude']
del testing_simple['end station latitude']
del testing_simple['end station longitude']
del testing_simple['starttime']
del testing_simple['stoptime']

In [ ]:
# export all data
training_clean.to_csv('data/training_clean.csv', index=False)
training_simple.to_csv('data/training_simple.csv', index=False)
testing_clean.to_csv('data/testing_clean.csv', index=False)
testing_simple.to_csv('data/testing_simple.csv', index=False)